# **Neural Grammar Dependency Parsing**
In this notebook, a neural transition-based dependency parser is built based off the paper <a href="https://nlp.stanford.edu/pubs/emnlp2014-depparser.pdf">A Fast and Accurate Dependency Parser using Neural Networks</a>.

The setup for a dependency parser is somewhat more sophisticated than tasks like classification or translation.


# **1. Prepare Data**




In [4]:
import numpy as np
from spacy import displacy
import random

## Read & Visualize Data
Here are some links about the data:
* A Universal Dependencies dataset from http://universaldependencies.org/docsv1/, specifically using UD_English dataset in version 1.4.
* Refer to https://universaldependencies.org/ for more about the Universal Dependencies framework in general.
* The data license can be found here: https://lindat.mff.cuni.cz/repository/xmlui/page/licence-UD-1.4


In [5]:
def load_data():
    train_set = cp.load(urlopen("https://drive.google.com/uc?export=download&id=1N4B4bC4ua0bFMIYeNdtNQQ72LxxKLPas"))
    test_set = cp.load(urlopen("https://drive.google.com/uc?export=download&id=1TE2AflhABbz41dLMGmD1kTm7WevYpU31"))
    return train_set, test_set

In [6]:
if __name__ == '__main__':
    train_set, test_set = load_data()
    print("Num. Train Examples:", len(train_set))
    print("Num. Test Examples: ", len(test_set))

Num. Train Examples: 12543
Num. Test Examples:  2002


Next, the training data is visualized, which contains labeled dependency trees. At test time, the goal will be to predict the dependency arcs & labels for a given sentence.

In [7]:
def display_sentence(sent):
    res = {'words': [{'text': "<ROOT>", 'tag': 'POS_ROOT'}], 'arcs': []}
    for i in range (len(sent['word'])):
        res['words'].append({'text': sent['word'][i], 'tag': sent['pos'][i]})
        s = i + 1
        e = sent['head'][i]
        direc = "left"
        if s > e:
            s = sent['head'][i]
            e = i + 1
            direc = 'right'
        cur = {'start': s, 'end': e, 'label': sent['label'][i], 'dir': direc}
        res['arcs'].append(cur)
    displacy.render(res, style="dep", manual=True, jupyter=True, options={'distance': 70})


In [8]:
if __name__ == '__main__':
    MIN_SENT_LEN, MAX_SENT_LEN = 4, 17
    for x in random.sample(list(filter(lambda x: len(x['word']) >= MIN_SENT_LEN and len(x['word']) <= MAX_SENT_LEN, train_set)), 5):
        display_sentence(x)

## Build Vocabulary

Next, the `Vocabulary` class is built. This maps each word, part-of-speech tag (POS), and label to an id (index), which we will later use in our embeddings. We also need to enumerate each possible transition, and map it to an id, since this is what our neural network will try to predict. The `Vocabulary` class does this as follows.

Suppose there are $n$ labels. For each label, Left-Arc (LA) and Right-Arc (RA) action for that label is created; and a separate Shift (S) action is created. This creates a total of $2n+1$ actions that our dependency parser will be able to choose from.

In [9]:
class Vocabulary(object):
    def __init__(self, dataset):

        UNK = '<UNK>'
        NULL = '<NULL>'
        ROOT = '<ROOT>'

        # Find the label of the root
        root_labels = list([l for ex in dataset for (h, l) in zip(ex['head'], ex['label']) if h == 0])
        assert len(set(root_labels)) == 1
        self.root_label = root_labels[0]

        # Create mapping from transitions to ids
        labels = sorted(list(set([w for ex in dataset for w in ex['label'] if w != self.root_label]))) # list of unique non-root labels
        labels = [self.root_label] + labels # add root label too
        self.n_labels = len(labels)

        transitions = ['LA-' + l for l in labels] + ['RA-' + l for l in labels] + ['S']
        self.n_trans = len(transitions) # 2*n_labels + 1
        self.tran2id = {t: i for (i, t) in enumerate(transitions)}
        self.id2tran = {i: t for (i, t) in enumerate(transitions)}

        # Create mapping from word, pos, & label to id
        # Do labels first
        self.LABEL_PREFIX = '<l>:'
        self.tok2id = {self.LABEL_PREFIX + l: i for (i, l) in enumerate(labels)}
        self.LABEL_NULL = self.tok2id[self.LABEL_PREFIX + NULL] = len(self.tok2id) # Add null label in

        # Do pos's
        self.POS_PREFIX = '<p>:'
        all_pos = sorted(set([self.POS_PREFIX + w for ex in dataset for w in ex['pos']])) # Get pos's
        self.tok2id.update({w: index + len(self.tok2id) for (index, w) in enumerate(all_pos)}) # Add poses in
        self.POS_NULL = self.tok2id[self.POS_PREFIX + NULL] = len(self.tok2id) # Add null pos
        self.POS_ROOT = self.tok2id[self.POS_PREFIX + ROOT] = len(self.tok2id) # Add root pos
        self.n_pos = 2 + len(all_pos) # +3 for null, root

        # Do words
        all_word = sorted(set([w for ex in dataset for w in ex['word']]))
        self.tok2id.update({w: index + len(self.tok2id) for (index, w) in enumerate(all_word)}) # Add words in
        self.WORD_UNK = self.tok2id[UNK] = len(self.tok2id) # Add unk word
        self.WORD_NULL = self.tok2id[NULL] = len(self.tok2id) # Add null word
        self.WORD_ROOT = self.tok2id[ROOT] = len(self.tok2id) # Add root word
        self.n_words = 3 + len(all_word) # +3 for unk, null, root

        self.id2tok = {v: k for (k, v) in self.tok2id.items()} # Flip it
        self.n_tokens = len(self.tok2id)

    def printStats(self):
        print('Num. labels:', self.n_labels)
        print('Num. transitions (2*n_labels + 1):', self.n_trans)
        print('Num. pos:', self.n_pos)
        print('Num. words:', self.n_words)
        print('Num. tokens:', self.n_tokens)


    def buildSentences(self, examples):
        processed_sentences = []
        for ex in examples:
            # Initialize words & dependencies
            words = [Word('<ROOT>', '<POS_ROOT>', 0, self.WORD_ROOT, self.POS_ROOT)]
            deps = []

            # Loop over words in sentence
            for i  in  range(len(ex['word'])):
                w = ex['word'][i]
                word_id = (self.tok2id[w] if w in self.tok2id else self.WORD_UNK)
                pos = self.POS_PREFIX + ex['pos'][i]
                pos_id = self.tok2id[pos]
                word = Word(ex['word'][i], ex['pos'][i],i+1, word_id, pos_id)

                words.append(word)
                deps.append((ex['head'][i], word, ex['label'][i] ))

            # Create dependencies
            dependencies = Dependencies(self)
            for dep in deps:
                dependencies.add(words[dep[0]], dep[1], dep[2])

            processed_sentences.append((words, dependencies))

        return processed_sentences

See some stats from the vocabulary.

In [10]:
Vocabulary(train_set).printStats()

Num. labels: 47
Num. transitions (2*n_labels + 1): 95
Num. pos: 52
Num. words: 16635
Num. tokens: 16735


# **2. Parser Data Structures**
In this section, some useful data structures for dependency parsing are defined. In particular, the `Stack` and `Buffer` structures are implemented, which make up a `ParserConfiguration`.

## Helpful Data Structures

First, some useful data classes are defined.

In [11]:
class Word(object):
    '''
    Represents a word in the sentence.
    '''

    def __init__(self, word, pos, idx, word_id=None, pos_id=None):
        self.word = word
        self.pos = pos
        self.idx = idx
        self.word_id = word_id
        self.pos_id = pos_id

    def __str__(self):
        return 'Word(idx=' + str(self.idx) + ", word='" + self.word+"', pos='"+self.pos+"', word_id="+str(self.word_id)+', pos_id='+ str(self.pos_id) +')'

    def copy(self):
        return Word(self.word, self.pos, self.idx, self.word_id, self.pos_id)

    def __eq__(self, obj):
        if not isinstance(obj, Word): return False
        if obj.idx == self.idx:
            assert obj.word == self.word and obj.pos == self.pos and obj.word_id == self.word_id and obj.pos_id == self.pos_id, 'Your Word object has been corrupted.'
        return obj.idx == self.idx


class Arc(object):
    '''
    Represents an arc between two words.
    '''

    def __init__(self, head, dependent, label, label_id):
        self.head=head # Word object
        self.dependent=dependent # Word object
        self.label=label
        self.label_id = label_id

    def __str__(self):
        return 'Arc(head_idx='+str(self.head.idx)+', dep_idx='+str(self.dependent.idx)+', label_id='+ str(self.label_id)+')'


class Dependencies(object):
    '''
    Represents the dependency arcs in a sentence.
    '''

    def __init__(self, vocab):
        self.arcs = []
        self.vocab = vocab
        self.dep_to_head_mapping = {} # For fast lookup

    def add(self, head, dependent, label):
        '''
        Add a dependency from head to dependent with label.
        Inputs:
            head: Word object
            dependent: Word object
            label: str
        '''
        assert label[:3] != 'LA-' and label[:3] != 'RA-', 'You need to pass in just the label to add(...), not the entire action.'
        assert head is not None and dependent is not None, "You must pass in two Word objects to add(...)."
        self.arcs.append(Arc(head, dependent, label, self.vocab.tok2id[self.vocab.LABEL_PREFIX+label]))
        assert dependent.idx not in self.dep_to_head_mapping
        self.dep_to_head_mapping[dependent.idx] = self.arcs[-1]

    def getArcToHead(self, dependent):
        '''
        Returns the Arc that connects the head of dependent to dependent.
        Inputs:
            dependent: Word object
        '''
        if dependent.idx == 0: # Special case for ROOT
            return Arc(None, dependent, None, None)
        return self.dep_to_head_mapping[dependent.idx]

    def __iter__(self):
        return iter(self.arcs) # Allows you to iterate "for x in Dependencies(...)"

## Stack & Buffer
Here, the stack and buffer data structures are defined.

In [12]:
class Stack(object):
    def __init__(self, input=[]):
        self.stack = [word.copy() for word in input]

    def push(self, item):
        self.stack.append(item)

    def pop(self):
        assert len(self.stack) > 0

        return self.stack.pop()

    def get_si(self, i):
        assert i > 0, 'Must provide i > 0'
        return self.stack[-i] if len(self.stack) >= i else None

    def __getitem__(self, idx):
        return self.stack[idx]

    def __len__(self):
        return len(self.stack)

    def __str__(self):
        return str([str(x) for x in self.stack])


class Buffer(object):
    def __init__(self, sentence):
        self.buffer = [word.copy() for word in sentence]

    def pop(self):
        assert len(self.buffer) > 0

        item = self.buffer[0]
        self.buffer = self.buffer[1:]
        return item

    def get_bi(self, i):
        assert i > 0, 'Must provide i > 0'
        return self.buffer[i-1] if len(self.buffer) >= i else None

    def __getitem__(self, idx):
        return self.buffer[idx]

    def __len__(self):
        return len(self.buffer)

    def __str__(self):
        return str([str(x) for x in self.buffer])

# **3. Parser Configuration**

Next, a `ParserConfiguration` class is created, which contains the `Stack`, `Buffer`, and `Dependencies` data structures.

Let $\sigma$ represent the stack, $\beta$ the buffer, and $A$ the set of arcs (dependencies). Based on the value of `transition`, `parse_step(self, transition)` does the following:
* `transition = 'S'`: &nbsp;<b>Shift</b> $w_k$ from the buffer to the stack. $(\sigma, w_k|\beta, A) \Rightarrow (\sigma|w_k, \beta, A)$
* `transition = 'LA-label'`: &nbsp;Add a <b>left arc</b> with label $label$ from $w_j$ to $w_i$. $(\sigma |w_i w_j , \beta, A) \Rightarrow (\sigma |w_j, \beta, A \cup \{(w_j, label, w_i)\})$
* `transition = 'RA-label'`: &nbsp;Add a <b>right arc</b> with label $label$ from $w_i$ to $w_j$. $(\sigma |w_i w_j , \beta, A) \Rightarrow (\sigma |w_i, \beta, A \cup \{(w_i, label, w_j)\})$


In [15]:
class ParserConfiguration(object):
    def __init__(self, sentence, vocab):
        self.vocab = vocab

        assert sentence[0].word_id == self.vocab.WORD_ROOT
        self.stack = Stack([sentence[0]]) # Initialize stack with ROOT
        self.buffer = Buffer(sentence[1:]) # Initialize buffer with sentence
        self.dependencies = Dependencies(vocab)

    def parse_step(self, transition):
        assert transition in self.vocab.tran2id

        if transition == "S":
          wk = self.buffer.pop()
          self.stack.push(wk)
        elif transition[:2] == "LA":
          label = transition[3:]
          wj = self.stack.pop()
          wi = self.stack.pop()
          self.dependencies.add(wj, wi, label)
          self.stack.push(wj)
        else:
          label = transition[3:]
          wj = self.stack.pop()
          wi = self.stack.pop()
          self.dependencies.add(wi, wj, label)
          self.stack.push(wi)


# **4. Generate Training Data**
The dataset contains many sentences along with their gold dependency parses. In order to use a transition-based parsing algorithm, the next parser action at each time step is predicted, based on the current parser configuration. Thus, each sentence is transformed into a series of training examples, where the input features are based on the current parser configuration and the correct label is the gold action.

In this section, an algorithm to select the next parser action at each time step based on the gold dependency parse is implemented. Then, features from each step's parser configuration are extracted, which the neural network will use to predict the next action.

##  Compute Gold Action

Next, the function `get_gold_action(stack, buffer, gold_dependencies)` is defined. Given a stack and buffer, this function returns the next action of the parser based on the gold dependencies.

Let $s_i$ be $i$th element of the stack and $h(s_i)$ be the head word of $s_i$. The pseudocode is as follows:

1. If the stack only contains `ROOT`:
 - If the buffer is not empty, return `S` (shift).
 - If the buffer is empty, return `DONE`, indicating the parse is complete.
2. If $h(s_2)=s_1$, return `LA-label`. Here, `label` is the label of the arc that attaches $s_2$ to $s_1$.
3. If $h(s_1)=s_2$ <b>and</b> $h(b_i) \neq s_1$ for all words $b_i$ in the buffer, return `RA-label`. Here, `label` is the label of the arc that attaches $s_1$ to $s_2$.
 - This condition means that you cannot attach $s_1$ until everything in the buffer that depends on $s_1$ is attached. You should think about why this condition is necessary!
4. Otherwise:
 - If the buffer is not empty, return `S`.
 - If the buffer is empty, return `None`, indicating a failed parse (i.e., the sentence is non-projective).


In [16]:
def get_gold_action(stack, buffer, gold_dependencies):
    action = None

    s1 = stack.get_si(1)
    #ROOT = "<ROOT>"
    if s1.word == '<ROOT>':
      if buffer.get_bi(1):
        return "S"
      else:
        return "DONE"

    s2 = stack.get_si(2)
    label_of_arc_2 = gold_dependencies.getArcToHead(s2)
    if label_of_arc_2.head == s1:
      return "LA-" + label_of_arc_2.label

    label_of_arc_1 = gold_dependencies.getArcToHead(s1)
    if label_of_arc_1.head == s2:
      action = "RA-" + label_of_arc_1.label

      for i in range(len(buffer)):
        if gold_dependencies.getArcToHead(buffer.get_bi(i + 1)).head == s1:
          action = None
          break

      if action:
        return action

    if buffer.get_bi(1):
      return "S"


    return action

## Generate Training Examples
Now a function to generate the training examples is defined. Recall that each sentence needs to be converted into a series of separate training examples. Each training example will essentially be a partial parser configuration along with its gold action; the goal of the neural network will be to predict this action from the parser configuration.

In [18]:
def generate_training_examples(sentence, gold_dependencies, vocab, feat_extract = lambda parser_config: []):
    '''
    Create training instances for sentence.
    Inputs:
        sentence: list of Word objects
        gold_dependencies: Dependencies object that contains the complete gold dependency tree
        vocab: Vocabulary object
        feat_extract: Feature extraction function
    Outputs:
        training_examples: List of tuples (features, label), where features is a list and label is a string
    steps:
        (1) Initialize the parser configuration
        (2) Repeatedly call get_gold_action(...) on current parser confuration until the gold action is 'DONE'
        (3) If the gold action is None at any step, return []  (indicating the sentence cannot be parsed; it is non-projective)
        (4) Otherwise, append tuple (features, gold action) to training_examples, where features is the result of calling feat_extract on your parser configuration
        (5) Update the parser configuration according to the gold action
        (6) Return training_examples
    '''

    training_examples = []
    parser_config = ParserConfiguration(sentence, vocab)

    while True:
      action = get_gold_action(parser_config.stack, parser_config.buffer, gold_dependencies)
      if action == "DONE":
        break
      if not action:
        return []

      #(features, gold action)
      training_examples.append((feat_extract(parser_config), action))

      #update parser configuration
      parser_config.parse_step(action)

    return training_examples

The following function calls `generate_training_examples(...)` on every sentence in the dataset to create the full training data. You do <b>not</b> need to edit it.

In [20]:
def generate_all_training_examples(vocab, sentences, feat_extract = lambda parser_config: []):
    '''
    Generate training examples for all sentences.
    '''
    all_training_examples = []
    successful_sents = 0
    for sentence in sentences:
        training_examples = generate_training_examples(sentence[0], sentence[1], vocab, feat_extract)
        if training_examples != []:
            all_training_examples += training_examples
            successful_sents += 1

    print("Successfully generated training examples for", successful_sents, "/", len(sentences), "sentences")
    print("Number of training examples:", len(all_training_examples))

    return all_training_examples

In [21]:
if __name__ == '__main__':
    _vocab = Vocabulary(train_set) # Variable just used in this cell
    generate_all_training_examples(_vocab, _vocab.buildSentences(train_set))

Successfully generated training examples for 11888 / 12543 sentences
Number of training examples: 373100


## Extract Features
So far, individual training instances can be created. Each instance is made up of a parser configuration along with the gold action that the classifier should be trained to predict.

In order to make this prediction, the neural network relies on features extracted from each parser configuration. This follows the procedure described at the end of Section 3.1 of <a href="https://nlp.stanford.edu/pubs/emnlp2014-depparser.pdf">A Fast and Accurate Dependency Parser using Neural Networks</a>.

In total, we will extract 48 features from the parser configuration $-$ 18 word features, 18 POS features, and 12 label features:
* Word & POS features for $s_1$, $s_2$, $s_3$ (top 3 items of the stack)
* Word & POS features for $b_1$, $b_2$, $b_3$ (top 3 items of the buffer)
* Word, POS, & label features for $lc_1(s_1)$, $lc_2(s_1)$, $lc_1(s_2)$, $lc_2(s_2)$ (the first & second leftmost children of the top 2 items on the stack)
* Word, POS, & label features for $rc_1(s_1)$, $rc_2(s_1)$, $rc_1(s_2)$, $rc_2(s_2)$ (the first & second rightmost children of the top 2 items on the stack)
* Word, POS, & label features for $lc_1(lc_1(s_1))$, $lc_1(lc_1(s_2))$, $rc_1(rc_1(s_1))$, $rc_1(rc_1(s_2))$ (the leftmost of the leftmost & rightmost of the rightmost children of the top 2 items on the stack)

A separate function for each of the five extractions above are defined. Each function will return a list of word features, a list of POS features, and a list of label features.

A "feature" refers to the id (index in the Vocabulary) of a word, POS, or label. The neural network will then construct embeddings for each id.

First, write 2 functions to extract features corresponding to the words at the top of the stack & buffer:
* `get_top3_stack_features(parser_config)` <b>[3 points]</b>: Return word & POS features (ids) for $s_1$, $s_2$, $s_3$ (top 3 words on the stack).
* `get_top3_buffer_features(parser_config)` <b>[3 points]</b>: Return word & POS features (ids) for $b_1$, $b_2$, $b_3$ (top 3 words on the buffer).

Wherever a particular word does not exist (such as when the stack or buffer has length $< 3$) use the appropriate NULL token. This is necessary to ensure that our neural network will see an equally sized feature vector for each example.

In [22]:
def get_top3_stack_features(parser_config):
    '''
    Get the word and POS features for s1, s2, and s3 (the top 3 items on the stack)
    Returns:
        word_features: List of word ids for s1, s2, s3 (use vocab.WORD_NULL if a word does not exist)
        pos_features: List of POS ids for s1, s2, s3 (use vocab.POS_NULL if a word does not exist)
    '''
    word_features, pos_features = [parser_config.vocab.WORD_NULL]*3, [parser_config.vocab.POS_NULL]*3

    for i in range(3):
      si = parser_config.stack.get_si(i + 1)
      if si:
        word_features[i] = si.word_id
        pos_features[i] = si.pos_id

    return word_features, pos_features

In [24]:
def get_top3_buffer_features(parser_config):
    '''
    Get the word and POS features for b1, b2, and b3 (the top 3 items on the buffer)
    Returns:
        word_features: List of word ids for b1, b2, b3 (use vocab.WORD_NULL if a word does not exist)
        pos_features: List of POS ids for b1, b2, b3 (use vocab.POS_NULL if a word does not exist)
    '''
    word_features, pos_features = [parser_config.vocab.WORD_NULL]*3, [parser_config.vocab.POS_NULL]*3

    for i in range(3):
      si = parser_config.buffer.get_bi(i + 1)
      if si:
        word_features[i] = si.word_id
        pos_features[i] = si.pos_id

    return word_features, pos_features

The remaining features have to do with the leftmost & rightmost children of the words at the top of the stack & buffer. Write the following 2 helper functions to make it easier to access these dependents:
* `get_lc(parser_config, word)`: Return a list of arcs to dependents of `word`, sorted from <b>left to right</b>. Only include dependents that are to the <b>left</b> of `word` in the sentence.
* `get_rc(parser_config, word)`: Return a list of arcs to dependents of `word`, sorted from <b>right to left</b>. Only include dependents that are to the <b>right</b> of `word` in the sentence.

In [26]:
def get_lc(parser_config, word):
    '''
    Finds the left dependents of word, sorted from left to right.
    Returns:
        A list of Arcs whose head is word, sorted by the indices of the dependent word from left to right.
    '''
    arc_list = parser_config.dependencies
    arcs = []
    for arc in arc_list:
      if arc.head == word and arc.dependent.idx < word.idx:
        arcs.append(arc)
    arcs.sort(key=lambda arc: arc.dependent.idx)

    return arcs

In [28]:
def get_rc(parser_config, word):
    '''
    Finds the right dependents of word, sorted from right to left.
    Returns:
        A list of Arcs whose head is word, sorted by the indices of the dependent word from right to left.
    '''
    arc_list = parser_config.dependencies
    arcs = []
    for arc in arc_list:
      if arc.head == word and arc.dependent.idx > word.idx:
        arcs.append(arc)
    arcs.sort(reverse=True, key=lambda arc: arc.dependent.idx)

    return arcs

Let $lc_j(s_i)$ be the $j$th leftmost child of the $i$th item on the stack. Write the following function:
* `get_lc1_lc2_features(parser_config, i)`: Return word & POS features for $lc_1(s_i)$ and $lc_2(s_i)$. Additionally, return label features (the label ids) for the arcs that attach $lc_1(s_i)$ to $s_i$ and $lc_2(s_i)$ to $s_i$. As before, wherever a particular word does not exist, use the appropriate NULL token.

We will call this function with `i=1` and `i=2`, accounting for the words $lc_1(s_1)$, $lc_2(s_1)$, $lc_1(s_2)$, $lc_2(s_2)$.

In [29]:
def get_lc1_lc2_features(parser_config, i):
    '''
    Get the word, POS, and label features for lc1(si) and lc2(si), where i in {1, 2}
    Returns:
        word_features: List of word ids for lc1(si), lc2(si) (use vocab.WORD_NULL if a word does not exist)
        pos_features: List of POS ids for lc1(si), lc2(si) (use vocab.POS_NULL if a word does not exist)
        label_features: List of label ids for lc1(si), lc2(si) (use vocab.LABEL_NULL if a word does not exist)
    '''
    assert i in {1,2}
    word_features, pos_features, label_features = [parser_config.vocab.WORD_NULL]*2, [parser_config.vocab.POS_NULL]*2, [parser_config.vocab.LABEL_NULL]*2

    si = parser_config.stack.get_si(i)
    lc = get_lc(parser_config, si)
    if len(lc) >= 1:
      lc_1 = lc[0]
      word_features[0] = lc_1.dependent.word_id
      pos_features[0] = lc_1.dependent.pos_id
      label_features[0] = lc_1.label_id

    if len(lc) >= 2:
      lc_2 = lc[1]
      word_features[1] = lc_2.dependent.word_id
      pos_features[1] = lc_2.dependent.pos_id
      label_features[1] = lc_2.label_id

    return word_features, pos_features, label_features

The analagous function for the rightmost children is implemented below. Let $rc_j(s_i)$ be the $j$th rightmost child of the $i$th item on the stack. Write the following function:
* `get_rc1_rc2_features(parser_config, i)` : Return word & POS features for $rc_1(s_i)$ and $rc_2(s_i)$. Additionally, return label features (the label ids) for the arcs that attach $rc_1(s_i)$ to $s_i$ and $rc_2(s_i)$ to $s_i$. As before, wherever a particular word does not exist, use the appropriate NULL token.

We will call this function with `i=1` and `i=2`, accounting for the words $rc_1(s_1)$, $rc_2(s_1)$, $rc_1(s_2)$, $rc_2(s_2)$.

In [31]:
def get_rc1_rc2_features(parser_config, i):
    '''
    Get the word, POS, and label features for rc1(si) and rc2(si), where i in {1, 2}
    Returns:
        word_features: List of word ids for rc1(si), rc2(si) (use vocab.WORD_NULL if a word does not exist)
        pos_features: List of POS ids for rc1(si), rc2(si) (use vocab.POS_NULL if a word does not exist)
        label_features: List of label ids for rc1(si), rc2(si) (use vocab.LABEL_NULL if a word does not exist)
    '''
    assert i in {1,2}
    word_features, pos_features, label_features = [parser_config.vocab.WORD_NULL]*2, [parser_config.vocab.POS_NULL]*2, [parser_config.vocab.LABEL_NULL]*2

    si = parser_config.stack.get_si(i)
    rc = get_rc(parser_config, si)
    if len(rc) >= 1:
      rc_1 = rc[0]
      word_features[0] = rc_1.dependent.word_id
      pos_features[0] = rc_1.dependent.pos_id
      label_features[0] = rc_1.label_id

    if len(rc) >= 2:
      rc_2 = rc[1]
      word_features[1] = rc_2.dependent.word_id
      pos_features[1] = rc_2.dependent.pos_id
      label_features[1] = rc_2.label_id

    return word_features, pos_features, label_features

Finally, the last function is implemented to bring it aqll together:
* `get_llc_rrc_features(parser_config, i)`: Return word & POS features for $lc_1(lc_1(s_i))$ and $rc_1(rc_1(s_i))$. Additionally, return label features (the label ids) for the arcs that attach $lc_1(lc_1(s_i))$ to $lc_1(s_i)$ and $rc_1(rc_1(s_i))$ to $rc_1(s_i)$. As before, wherever a particular word does not exist, use the appropriate NULL token.

We will call this function with `i=1` and `i=2`, accounting for the words $lc_1(lc_1(s_1))$, $lc_1(lc_1(s_2))$, $rc_1(rc_1(s_1))$, $rc_1(rc_1(s_2))$.

In [34]:
def get_llc_rrc_features(parser_config, i):
    '''
    Get the word, POS, and label features for lc1(lc1(si)), and rc1(rc1(si)), where i in {1, 2}
    Returns:
        word_features: List of word ids for lc1(lc1(si)), and rc1(rc1(si)) (use vocab.WORD_NULL if a word does not exist)
        pos_features: List of POS ids for lc1(lc1(si)), and rc1(rc1(si)) (use vocab.POS_NULL if a word does not exist)
        label_features: List of label ids for lc1(lc1(si)), and rc1(rc1(si)) (use vocab.LABEL_NULL if a word does not exist)
    '''
    assert i in {1,2}
    word_features, pos_features, label_features = [parser_config.vocab.WORD_NULL]*2, [parser_config.vocab.POS_NULL]*2, [parser_config.vocab.LABEL_NULL]*2

    si = parser_config.stack.get_si(i)
    in_lc = get_lc(parser_config, si)

    if len(in_lc) >= 1:
      in_lc_1 = in_lc[0]
      out_lc = get_lc(parser_config, in_lc_1.dependent)
      if len(out_lc) >= 1:
        lc_1 = out_lc[0]
        word_features[0] = lc_1.dependent.word_id
        pos_features[0] = lc_1.dependent.pos_id
        label_features[0] = lc_1.label_id


    in_rc = get_rc(parser_config, si)

    if len(in_rc) >= 1:
      in_rc_1 = in_rc[0]
      out_rc = get_rc(parser_config, in_rc_1.dependent)
      if len(out_rc) >= 1:
        rc_1 = out_rc[0]
        word_features[1] = rc_1.dependent.word_id
        pos_features[1] = rc_1.dependent.pos_id
        label_features[1] = rc_1.label_id


    return word_features, pos_features, label_features

The function `extract_features(parser_config)` is defined to call each of these functions and returns a list of the 48 total features.

In [35]:
def extract_features(parser_config): # for both train & inference
    word_features, pos_features, label_features = [], [], []

    # 1. Get word & pos features for s1, s2, and s3
    (x, y) = get_top3_stack_features(parser_config)
    word_features, pos_features = word_features + x, pos_features + y


    # 2. Get word & pos features for b1, b2, and b3
    (x, y) = get_top3_buffer_features(parser_config)
    word_features, pos_features = word_features + x, pos_features + y


    # 3. Get word & pos & label features for lc1(s1), lc1(s2), lc2(s1), lc2(s2)
    (x, y, z) = get_lc1_lc2_features(parser_config, 1)
    word_features, pos_features, label_features = word_features + x, pos_features + y, label_features + z

    (x, y, z) = get_lc1_lc2_features(parser_config, 2)
    word_features, pos_features, label_features = word_features + x, pos_features + y, label_features + z


    # 4. Get word & pos & label features for rc1(s1), rc1(s2), rc2(s1), rc2(s2)
    (x, y, z) = get_rc1_rc2_features(parser_config, 1)
    word_features, pos_features, label_features = word_features + x, pos_features + y, label_features + z

    (x, y, z) = get_rc1_rc2_features(parser_config, 2)
    word_features, pos_features, label_features = word_features + x, pos_features + y, label_features + z


    # 5. Get word & pos & label features for lc1(lc1(s1)), lc1(lc1(s2)), rc1(rc1(s1)), rc1(rc1(s2))
    (x, y, z) = get_llc_rrc_features(parser_config, 1)
    word_features, pos_features, label_features = word_features + x, pos_features + y, label_features + z

    (x, y, z) = get_llc_rrc_features(parser_config, 2)
    word_features, pos_features, label_features = word_features + x, pos_features + y, label_features + z


    features = word_features + pos_features + label_features
    assert len(features) == 48
    return features

# **5. Dataset & Model**
Now the Pytorch `TrainDataset` class and model are defined.

In [37]:
import torch
import torch.nn as nn
import torch.nn.functional as F

## Instantiate Dataset
A Pytorch `TrainDataset` class is defined, which is used to feed the training data to the model.

In [38]:
class TrainDataset(torch.utils.data.Dataset):
    def __init__(self, data, vocab):
        self.X = np.array([d[0] for d in data])
        self.y = np.array([vocab.tran2id[d[1]] for d in data])

    def __getitem__(self, index):
        return self.X[index], self.y[index]

    def __len__(self):
        return len(self.X)

##  Define Model

Here a feed-forward network is defined with an embedding layer, a single hidden layer, and an output layer. The `forward(...)` method takes in the features extracted from the parser configuration and predict the next parser action.

In [39]:
class Model(nn.Module):
    def __init__(self, num_embeddings, embed_size, n_features, hidden_size, n_classes, dropout_prob):
        '''
        Initialize the weights of feed-forward neural network.
        Args:
            num_embeddings: Number of embedding vectors in embedding layer (int)
            embed_size: Size of the embedding vectors in embedding layer (int)
            n_features: Number of features in the input to the model (int)
            hidden_size: Hidden size (int)
            n_classes: Number of classes in output (int)
            dropout_prob: Probability of dropout (float)
        '''
        super(Model, self).__init__()

        self.embeddings = nn.Embedding(num_embeddings, embed_size)
        self.linear = nn.Linear(n_features * embed_size, hidden_size)
        self.dropout = nn.Dropout(dropout_prob)
        self.linear_output = nn.Linear(hidden_size, n_classes)


    def forward(self, x):
        '''
        This function predicts the next parser action, given the features extracted from the current parser state.
        Inputs:
             x: input features, [batch_size, n_features]
        Returns:
            logits: [batch_size, n_classes]
        '''

        embeddding = self.embeddings(x)
        dims = list(embeddding.size())
        embeddding = torch.reshape(embeddding, (dims[0], dims[1] * dims[2]))

        hidden = self.linear(embeddding)
        hidden = F.relu(hidden)

        dropout_vec= self.dropout(hidden)

        res = self.linear_output(dropout_vec)

        return res

# **6: Train Model**

Finally, the model is ready to be trained.

In [41]:
import math
from torch import optim
from tqdm.notebook import tqdm
import time

First, we read in the training dataset, create training examples, extract features, and instantiate the Pytorch `Dataset`.

In [42]:
def prepare_data(train_name='train', test_name='test'):
    train_set, test_set = load_data()

    vocab = Vocabulary(train_set)
    vocab.printStats()
    print()

    train_set = vocab.buildSentences(train_set)
    test_set = vocab.buildSentences(test_set)

    train_examples = generate_all_training_examples(vocab, train_set, feat_extract=extract_features)

    return vocab, train_examples, test_set

if __name__== "__main__":
    vocab, train_examples, test_data = prepare_data()
    train_dataset = TrainDataset(train_examples, vocab)

Num. labels: 47
Num. transitions (2*n_labels + 1): 95
Num. pos: 52
Num. words: 16635
Num. tokens: 16735

Successfully generated training examples for 11888 / 12543 sentences
Number of training examples: 373100


The neural network is trained using cross-entropy loss.

In [43]:
def train_model(model, vocab, train_data_loader, optimizer, n_epochs, device):
    loss_func = nn.CrossEntropyLoss()
    for epoch in range(n_epochs):
        start = time.time()
        n_batch = 0
        total_loss = 0
        model.train()
        for train_x, train_y in tqdm(train_data_loader):
            optimizer.zero_grad()
            train_x = train_x.to(device)
            train_y = train_y.to(device)
            logits = model(train_x)
            loss = loss_func(logits, train_y)
            loss.backward()
            optimizer.step()

            total_loss +=  loss.item()
            n_batch += 1

        print('Epoch:{:2d}/{}\t Loss: {:.4f} \t({:.2f}s)'.format(epoch + 1, n_epochs, total_loss / n_batch, time.time() - start))

Next we instantiate the model and an <a href=https://jmlr.org/papers/volume12/duchi11a/duchi11a.pdf>Adagrad</a> optimizer.

In [44]:
if __name__ == "__main__":
    # HYPERPARAMETERS - Feel free to change
    BATCH_SIZE = 1024
    LEARNING_RATE = 0.01
    N_EPOCHS = 10
    HIDDEN_SIZE = 300
    DROPOUT_PROB = 0.1
    EMBED_SIZE = 100
    WEIGHT_DECAY = 1e-8
    N_EMBEDDINGS = vocab.n_tokens
    N_FEATURES = 48
    N_CLASSES = vocab.n_trans #

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, drop_last=True, shuffle=True)
    model = Model(N_EMBEDDINGS, EMBED_SIZE, N_FEATURES, HIDDEN_SIZE, N_CLASSES, DROPOUT_PROB).to(device)
    optimizer = optim.Adagrad(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

Run the cell below to train the model.

In [45]:
if __name__=='__main__':
    train_model(model, vocab, train_data_loader, optimizer, N_EPOCHS, device)

  0%|          | 0/364 [00:00<?, ?it/s]

Epoch: 1/10	 Loss: 0.7661 	(4.65s)


  0%|          | 0/364 [00:00<?, ?it/s]

Epoch: 2/10	 Loss: 0.2713 	(3.53s)


  0%|          | 0/364 [00:00<?, ?it/s]

...

Epoch: 8/10	 Loss: 0.1565 	(4.39s)


  0%|          | 0/364 [00:00<?, ?it/s]

Epoch: 9/10	 Loss: 0.1504 	(4.18s)


  0%|          | 0/364 [00:00<?, ?it/s]

Epoch:10/10	 Loss: 0.1437 	(3.44s)


# **7. Evaluate Model**

Now that the model is trained, it can be used to parse unseen sentences from a test set.

## Select Best Legal Prediction
The two functions combined below takes a trained model and a batch of parser configurations, and returns the highest probability legal predicted action for each parser configuration.

In [46]:
def select_best_legal_action(parser_configs, predictions, n_labels):
    '''
    Returns the highest probability **legal** prediction for each parser configuration.
    Inputs:
        parser_configs: list of parser configurations of length N
        predictions: np.array of size [N, 2*n_labels + 1]
        n_labels: int, the number of labels in our model
    Returns:
        preds: np.array of length N, containing the indices of the highest probability legal action for each example
    '''
    for i in range(len(parser_configs)):
      parser_config = parser_configs[i]
      pred = predictions[i, ]
      l, left, right = 0, 0, 0
      stack, buff = len(parser_config.stack), len(parser_config.buffer)
      if buff > 0:
        l = 1
      if stack >= 3:
        left, right = 1, 1
      if stack == 2:
        right = 1
      if left == 1:
        left = np.ones((n_labels,))
      else:
        left = np.zeros((n_labels,))

      if right == 1:
        right = np.ones((n_labels,))
      else:
        right = np.zeros((n_labels,))

      maps = np.concatenate((left, right, [l]))
      predictions[i, ] = np.sum([maps*100000, pred], axis=0)

    preds = np.argmax(predictions, axis = 1) # Change this! This selects the highest probability action, regardless of legality.

    return preds

In [47]:
def predict(model, vocab, parser_configs):
    '''
    Predicts the next transition for each ParserConfiguration in the batch (`parsers`).
    '''
    model_device = next(model.parameters()).device

    x = np.array([extract_features(p) for p in parser_configs])
    x = torch.from_numpy(x).long().to(model_device)

    with torch.no_grad():
        pred = model(x)

    pred = pred.detach().cpu().numpy()
    pred = select_best_legal_action(parser_configs, pred, vocab.n_labels)
    actions = [vocab.id2tran[p] for p in pred]
    return actions

## Test Set Attachment Score
The following functions use the model to parse all sentences in the test set, and compute the attachment score. The** unlabeled attachment score** is the percentage of arcs in the test set for which the model gets the head correct. The **labeled attachment score** is the percentage of arcs for which the model gets both the head and the label correct. Thus, attachment score is a number between 0 and 100, and a higher score is better.

In [48]:
def run_inference(sentences, model, vocab, batch_size=2000):
    N = len(sentences)

    # Initialize parser configs
    parser_configs = [None] * N
    for i in range(N):
        sent = sentences[i]
        parser_config = ParserConfiguration(sent, vocab)
        parser_configs[i] = parser_config

    parses_completed = [False] * N # Indicates whether a given parse is completed

    while sum(parses_completed) != N:

        # Get batch along with indices
        batch_idxes = []
        for idx in range(N):
            if not parses_completed[idx]: batch_idxes.append(idx)
            if len(batch_idxes) == batch_size: break
        batch = [parser_configs[idx] for idx in batch_idxes]

        # Make prediction, run a parse step, and check for completion
        transitions = predict(model, vocab, batch)
        for idx, parser, transition in zip(batch_idxes, batch, transitions):
            parser.parse_step(transition)
            if not parser.buffer.buffer and len(parser.stack.stack) == 1:
                parses_completed[idx] = True

    return [parser.dependencies for parser in parser_configs]

def transform_to_head_label(dependencies):
    head = [-1] * len(dependencies.arcs)
    label = [-1] * len(dependencies.arcs)
    for dep in dependencies.arcs:
        head[dep.dependent.idx-1] = dep.head.idx
        label[dep.dependent.idx-1] = dep.label_id
    return head, label


def evaluate(model, vocab, dataset, eval_batch_size=5000):
    model.eval()
    sentences = [x[0] for x in dataset]
    gold_dependencies = [x[1] for x in dataset]
    pred_dependencies = run_inference(sentences, model, vocab, eval_batch_size)

    UAS, LAS = 0.0, 0.0

    all_tokens = 0

    for i in range(len(gold_dependencies)):
        assert len(gold_dependencies[i].arcs) == len(pred_dependencies[i].arcs)

        # Get gold answers
        gold_head, gold_label = transform_to_head_label(gold_dependencies[i])

        # Get predictions
        pred_head, pred_label = transform_to_head_label(pred_dependencies[i])

        assert len(gold_head) == len(pred_head) and len(gold_label) == len(pred_label)
        assert -1 not in gold_head + gold_label + pred_head + pred_label

        for pred_h, gold_h, pred_l, gold_l  in zip(pred_head, gold_head, pred_label, gold_label):
            UAS += (1 if pred_h == gold_h else 0)
            LAS += (1 if pred_h == gold_h and pred_l == gold_l else 0)
            all_tokens += 1
    return UAS / all_tokens * 100, LAS / all_tokens * 100, pred_dependencies

The attachment scores are computed below.

In [49]:
if __name__=="__main__":
    UAS, LAS, test_predictions = evaluate(model, vocab, test_data)
    print("Test Set Unlabeled Attachment Score:", UAS)
    print("Test Set Labeled Attachment Score:", LAS)

Test Set Unlabeled Attachment Score: 83.45792905996501
Test Set Labeled Attachment Score: 80.8573246381422


## Qualitative Analysis

Here the model can be analyzed qualitatively to get a feel for the strengths and shortcomings of the model.

Some example sentences from the test set are printed. For each sentence, the gold (correct) dependency tree, the dependency tree predicted by the model, and a diagnostic of the gold tree are displayed. The diagnostic tree annotates the edges of the gold tree as follows:
* ✓: Edges for which you predicted both the <b>correct head & label</b>
* ⍻: Edges for which you predicted the <b>correct head but incorrect label</b>
* ×: Edges that you do not have in your tree (i.e., you predicted the <b>incorrect head<b>).

In [50]:
def diagnose(sentence, gold, pred):
    word, pos = [x.word for x in sentence], [x.pos for x in sentence]
    gold_head, gold_label = transform_to_head_label(gold)
    pred_head, pred_label = transform_to_head_label(pred)
    gold_label = [gold.vocab.id2tok[x][4:] for x in gold_label]
    pred_label = [gold.vocab.id2tok[x][4:] for x in pred_label]

    diff = ["✓"] * len(pred_head)
    for i in range(len(pred_head)):
        if gold_head[i]!=pred_head[i]: diff[i] = "×"
        elif gold_label[i]!=pred_label[i]: diff[i] = "⍻"
    unlabeled_score=sum([x in {"✓","⍻"}  for x in diff]) / len(diff)
    score=sum([x == "✓" for x in diff]) / len(diff)

    print("Your dependency tree:")
    sent = {"word": word[1:], "pos": pos[1:], "label": pred_label, "head": pred_head}
    display_sentence(sent)

    print("Gold dependency tree:")
    sent["label"], sent["head"] = gold_label, gold_head
    display_sentence(sent)

    print("Diagnostic of the gold tree:")
    sent["label"], sent["head"] = diff, gold_head
    display_sentence(sent)
    print('Unlabeled Attachment Score for this sentence:', unlabeled_score*100)
    print('Labeled Attachment Score for this sentence:', score*100, '\n')

def diagnose_sentences(idxes, data, preds, min_len, max_len, num_to_print=5):
    print('-'*100, '\n')
    for i in random.sample(list(filter(lambda x: len(data[x][0]) >= min_len and len(data[x][0]) <= max_len, idxes)), num_to_print):
        diagnose(data[i][0], data[i][1], preds[i])
        print('-'*100, '\n')

In [51]:
if __name__== '__main__':
    MIN_SENT_LEN, MAX_SENT_LEN = 8, 17
    idxes=list(range(len(test_data)))
    diagnose_sentences(idxes, test_data, test_predictions, MIN_SENT_LEN, MAX_SENT_LEN, num_to_print=5)

---------------------------------------------------------------------------------------------------- 

Your dependency tree:


Gold dependency tree:


Diagnostic of the gold tree:


Unlabeled Attachment Score for this sentence: 87.5
Labeled Attachment Score for this sentence: 75.0 

---------------------------------------------------------------------------------------------------- 

Your dependency tree:


Gold dependency tree:


...

Diagnostic of the gold tree:


Unlabeled Attachment Score for this sentence: 54.54545454545454
Labeled Attachment Score for this sentence: 54.54545454545454 

---------------------------------------------------------------------------------------------------- 

Your dependency tree:


Gold dependency tree:


Diagnostic of the gold tree:


Unlabeled Attachment Score for this sentence: 100.0
Labeled Attachment Score for this sentence: 100.0 

---------------------------------------------------------------------------------------------------- 

